### Adding batteries

The goal is to add batteries to a system. The main idea is to take a Bus, i.e. a node, and add a battery there. The procedure is a bit straightforward but we will flesh it out here. To see an example, see Joseph McKinsey's `prepare_systems.jl` file or this [tutorial](https://github.com/NREL-SIIP/SIIPExamples.jl/blob/master/test/4_PowerSimulationsDynamics_examples/03_inverter_model.jl).

First let's load in two systems. The first the 5 bus system that already has batteries and a large RTS GMLC that has no batteries (which we will add).

In [3]:
using Pkg
# Activate environment that has ProgressiveHedging installed
Pkg.activate("..")

  Activating project at `/lustre/eaglefs/projects/pvb/cju/ProgressiveHedging.jl`


In [4]:
using PowerSimulations, PowerSystems, PowerSystemCaseBuilder
using Xpress, Dates
const PSY = PowerSystems
const PSI = PowerSimulations

┌ Info: Xpress: Found license file /nopt/nrel/apps/xpressmp/8.13.0/bin/xpauth.xpr
└ @ Xpress /home/cju/.julia/packages/Xpress/xOQbX/src/license.jl:44
┌ Info: Xpress: Development license detected.
└ @ Xpress /home/cju/.julia/packages/Xpress/xOQbX/src/license.jl:89


PowerSimulations

In [5]:
system = build_system(PSITestSystems, "modified_RTS_GMLC_DA_sys", time_series_directory = "/tmp/scratch")

┌ Info: Loaded time series from storage file existing=modified_RTS_GMLC_DA_sys_time_series_storage.h5 new=/tmp/scratch/jl_nElY2w compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
└ @ InfrastructureSystems /home/cju/.julia/packages/InfrastructureSystems/Oc56m/src/hdf5_time_series_storage.jl:100


System
┌───────────────────┬─────────────┐
│ Property          │ Value       │
├───────────────────┼─────────────┤
│ System Units Base │ SYSTEM_BASE │
│ Base Power        │ 100.0       │
│ Base Frequency    │ 60.0        │
│ Num Components    │ 500         │
└───────────────────┴─────────────┘

Static Components
┌──────────────────────────────┬───────┬────────────────────────┬───────────────
│ Type                         │ Count │ Has Static Time Series │ Has Forecast ⋯
├──────────────────────────────┼───────┼────────────────────────┼───────────────
│ Arc                          │ 109   │ false                  │ false        ⋯
│ Area                         │ 3     │ true                   │ true         ⋯
│ Bus                          │ 73    │ false                  │ false        ⋯
│ FixedAdmittance              │ 3     │ true                   │ true         ⋯
│ HydroDispatch                │ 1     │ true                   │ true         ⋯
│ Line                         │ 105   │ false                  │ false        ⋯
│ LoadZone                     │ 21    │ false                  │ false        ⋯
│ PowerLoad                    │ 51    │ true                   │ true         ⋯
│ RenewableDispatch            │ 29    │ true                   │ true         ⋯
│ RenewableFix                 │ 31    │ true                   │ true         ⋯
│ TapTransformer               │ 15    │ false                  │ false        ⋯
│ ThermalStandard              │ 54    │ false                  │ false        ⋯
│ VariableReserve{ReserveDown} │ 1     │ true                   │ true         ⋯
│ VariableReserve{ReserveUp}   │ 4     │ true                   │ true         ⋯
└──────────────────────────────┴───────┴────────────────────────┴───────────────
                                                                1 column omitted

Time Series Summary
┌──────────────────────────────────┬─────────────────────┐
│ Property                         │ Value               │
├──────────────────────────────────┼─────────────────────┤
│ Components with time series data │ 123                 │
│ Total StaticTimeSeries           │ 124                 │
│ Total Forecasts                  │ 124                 │
│ Resolution                       │ 60 minutes          │
│ First initial time               │ 2020-01-01T00:00:00 │
│ Last initial time                │ 2020-12-30T00:00:00 │
│ Horizon                          │ 48                  │
│ Interval                         │ 1440 minutes        │
│ Forecast window count            │ 365                 │
└──────────────────────────────────┴─────────────────────┘

In [6]:
sys_name = "/lustre/eaglefs/projects/pvb/5_bus_battery_testing/data/new_systems/pv=30_storagehours=10_wind=30/sys.json"
sysj = PSY.System(sys_name, time_series_directory = "/tmp/scratch")

┌ Info: Loaded time series from storage file existing=sys_time_series_storage.h5 new=/tmp/scratch/jl_rSobrh compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
└ @ InfrastructureSystems /home/cju/.julia/packages/InfrastructureSystems/Oc56m/src/hdf5_time_series_storage.jl:100
┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = magnitude
│   valid_range = voltage_limits
│   valid_info.ist_struct = Bus(1, node_a, BusTypes.PV = 3, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}())
└ @ InfrastructureSystems /home/cju/.julia/packages/InfrastructureSystems/Oc56m/src/validation.jl:222
┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = magnitude
│   valid_range = voltage_limits
│   valid_info.ist_struct = Bus(4, node_d, BusTypes.REF = 4, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{Strin

System
┌───────────────────┬─────────────┐
│ Property          │ Value       │
├───────────────────┼─────────────┤
│ System Units Base │ DEVICE_BASE │
│ Base Power        │ 100.0       │
│ Base Frequency    │ 60.0        │
│ Num Components    │ 31          │
└───────────────────┴─────────────┘

Static Components
┌───────────────────┬───────┬────────────────────────┬───────────────┐
│ Type              │ Count │ Has Static Time Series │ Has Forecasts │
├───────────────────┼───────┼────────────────────────┼───────────────┤
│ Arc               │ 6     │ false                  │ false         │
│ Area              │ 1     │ false                  │ false         │
│ Bus               │ 5     │ false                  │ false         │
│ GenericBattery    │ 1     │ false                  │ false         │
│ Line              │ 6     │ false                  │ false         │
│ LoadZone          │ 1     │ false                  │ false         │
│ PowerLoad         │ 3     │ true                   │ false         │
│ RenewableDispatch │ 3     │ true                   │ false         │
│ ThermalStandard   │ 5     │ false                  │ false         │
└───────────────────┴───────┴────────────────────────┴───────────────┘

Time Series Summary
┌──────────────────────────────────┬────────────┐
│ Property                         │ Value      │
├──────────────────────────────────┼────────────┤
│ Components with time series data │ 6          │
│ Total StaticTimeSeries           │ 6          │
│ Total Forecasts                  │ 0          │
│ Resolution                       │ 60 minutes │
└──────────────────────────────────┴────────────┘

### Finding available nodes

Next, let's check what buses/nodes are available in each of the systems.

In [7]:
collect(get_components(Bus, sysj))

5-element Vector{Bus}:
 Bus(1, node_a, BusTypes.PV = 3, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}())
 Bus(4, node_d, BusTypes.REF = 4, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}())
 Bus(5, node_e, BusTypes.PV = 3, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}())
 Bus(3, node_c, BusTypes.PV = 3, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}())
 Bus(2, node_b, BusTypes.PQ = 2, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}())

Let's see what components are available in `node_a` and `node_b`, since Joseph adds batteries to one of these two.

In [8]:
@show get_component(Bus, sysj, "node_a")

get_component(Bus, sysj, "node_a") = Bus(1, node_a, BusTypes.PV = 3, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}())


node_a (Bus):
   number: 1
   name: node_a
   bustype: BusTypes.PV = 3
   angle: 0.0
   magnitude: 0.01
   voltage_limits: (min = 0.95, max = 1.05)
   base_voltage: 100.0
   area: nothing (Area)
   load_zone: zone1 (LoadZone)
   ext: Dict{String, Any}()
   InfrastructureSystems.SystemUnitsSettings:
      base_value: 100.0
      unit_system: UnitSystem.DEVICE_BASE = 1

In [9]:
@show get_component(Bus, sysj, "node_b")

get_component(Bus, sysj, "node_b") = Bus(2, node_b, BusTypes.PQ = 2, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}())


node_b (Bus):
   number: 2
   name: node_b
   bustype: BusTypes.PQ = 2
   angle: 0.0
   magnitude: 0.01
   voltage_limits: (min = 0.95, max = 1.05)
   base_voltage: 100.0
   area: nothing (Area)
   load_zone: zone1 (LoadZone)
   ext: Dict{String, Any}()
   InfrastructureSystems.SystemUnitsSettings:
      base_value: 100.0
      unit_system: UnitSystem.DEVICE_BASE = 1

OK, so although we tie a battery to a specific bus/node, the bus/node does not contain that information. Perhaps one can think of the bus/node as an address only and a GenericBattery is a description of the battery + it's home address, aka, the chosen bus/node. Let's check out the battery.

In [10]:
collect(get_components(GenericBattery, sysj))

1-element Vector{GenericBattery}:
 GenericBattery(Storage_10, true, Bus(2, node_b, BusTypes.PQ = 2, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}()), PrimeMovers.BA = 1, 5.0, (min = 0.0, max = 10.0), 1.0, 0.0, (min = 0.0, max = 1.0), (min = 0.0, max = 1.0), (in = 0.85, out = 1.0), 0.0, nothing, 200.0, nothing, Service[], nothing, Dict{String, Any}())

Aha! We now see that thre is a battery and it's home address of `node_b` is included. Great!

Let's also see what's available in the larger system.

In [11]:
for (i, bus) in enumerate(collect(get_components(Bus, system)))
    println("Bus $(i) name: $(bus.name)")
end

Bus 1 name: Bacon
Bus 2 name: Cobb
Bus 3 name: Cole
Bus 4 name: Adams
Bus 5 name: Aston
Bus 6 name: Curie
Bus 7 name: Attlee
Bus 8 name: Bajer
Bus 9 name: Bates
Bus 10 name: Adler
Bus 11 name: Cabell
Bus 12 name: Bailey
Bus 13 name: Alder
Bus 14 name: Cary
Bus 15 name: Allen
Bus 16 name: Chuhsi
Bus 17 name: Barton
Bus 18 name: Comte
Bus 19 name: Bach
Bus 20 name: Caesar
Bus 21 name: Carew
Bus 22 name: Bain
Bus 23 name: Chase
Bus 24 name: Bell
Bus 25 name: Carrel
Bus 26 name: Ali
Bus 27 name: Caruso
Bus 28 name: Avery
Bus 29 name: Asser
Bus 30 name: Aubrey
Bus 31 name: Bardeen
Bus 32 name: Chifa
Bus 33 name: Alger
Bus 34 name: Bloch
Bus 35 name: Agricola
Bus 36 name: Astor
Bus 37 name: Aiken
Bus 38 name: Clay
Bus 39 name: Barlow
Bus 40 name: Caine
Bus 41 name: Bordet
Bus 42 name: Banks
Bus 43 name: Curtiss
Bus 44 name: Basov
Bus 45 name: Balzac
Bus 46 name: Camus
Bus 47 name: Chain
Bus 48 name: Caxton
Bus 49 name: Alber
Bus 50 name: Clark
Bus 51 name: Attar
Bus 52 name: Anna
Bus 53 name

This system should not have any batteries. Let's double check.

In [12]:
collect(get_components(GenericBattery, system))

GenericBattery[]

### Stability check before and after battery add

Before adding a battery and after, it's good to run a optimal power flow to make sure the system is "stable" (see [ref](https://github.com/NREL-SIIP/SIIPExamples.jl/blob/master/test/4_PowerSimulationsDynamics_examples/03_inverter_model.jl)). So let's make sure the new system is stable and let's run this diagnosis before and after the addition of a battery.

In [13]:
res = solve_powerflow(system)
res["bus_results"]

┌ Info: Validating connectivity with Goderya algorithm
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/utils/network_calculations/ybus_calculations.jl:308
┌ Info: The System has no islands
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/utils/network_calculations/ybus_calculations.jl:254
┌ Info: PowerFlow solve converged, the results are exported in DataFrames
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/utils/power_flow.jl:357
┌ Info: Voltages are exported in pu. Powers are exported in MW/MVAr.
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/utils/power_flow.jl:189


,bus_number,Vm,θ,P_gen,P_load,P_net,Q_gen,Q_load,Q_net
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,101,1.04777,-0.862755,0.0,108.0,-108.0,110.242,22.0,88.242
2,102,1.04783,-0.862282,0.0,97.0,-97.0,107.223,20.0,87.2227
3,103,0.983808,-0.783493,0.0,180.0,-180.0,0.0,37.0,-37.0
4,104,0.983463,-0.769658,0.0,74.0,-74.0,0.0,15.0,-15.0
5,105,1.00436,-0.784978,0.0,71.0,-71.0,0.0,14.0,-14.0
6,106,0.989394,-0.75807,0.0,136.0,-136.0,0.0,27.0211,-27.0211
7,107,1.03745,-0.729067,355.0,125.0,230.0,108.299,25.0,83.2991
8,108,0.987728,-0.743545,0.0,171.0,-171.0,0.0,35.0,-35.0
9,109,0.979653,-0.611487,0.0,175.0,-175.0,0.0,36.0,-36.0


Values seem reasonable. Now, **let's add the battery**!

Looking back at the available buses in this system, let's add it to Bus #4, "Adams".

In [14]:
# first, get the bus we want to add the battery to
bus=PSY.get_component(PSY.Bus, system, "Adams")

Adams (Bus):
   number: 102
   name: Adams
   bustype: BusTypes.PV = 3
   angle: -0.1364470483941135
   magnitude: 1.04783
   voltage_limits: (min = 0.95, max = 1.05)
   base_voltage: 138.0
   area: 1 (Area)
   load_zone: 12.0 (LoadZone)
   ext: Dict{String, Any}("neighbors" => Set([4, 6, 2, 1]))
   InfrastructureSystems.SystemUnitsSettings:
      base_value: 100.0
      unit_system: UnitSystem.SYSTEM_BASE = 0

The following is taken from Joseph McKinsey's code.

In [15]:
# possible battery hours. Let's do 24 hours.
# hour_arr = [[2], [4], [10], [24], [4, 10], [2, 10], [4, 24], [2, 10]]
hours = 24 # hrs
base_power=200 # u"MW"
efficiency = 0.85
name = "Storage_$hours"

bat = PSY.GenericBattery(;
    name = name,
    available = true,
    bus = bus,
    prime_mover = PSY.PrimeMovers.BA,
    initial_energy = hours / 2,
    state_of_charge_limits = (min = 0.0, max = hours),
    rating = 1.0,
    active_power = 0.0,
    input_active_power_limits = (min = 0.0, max = 1.0),
    output_active_power_limits = (min = 0.0, max = 1.0),
    efficiency = (in = efficiency, out = 1.0),
    reactive_power = 0.0,
    reactive_power_limits = nothing,
    base_power = base_power
)


Storage_24 (GenericBattery):
   name: Storage_24
   available: true
   bus: Adams (Bus)
   prime_mover: PrimeMovers.BA = 1
   initial_energy: 2400.0
   state_of_charge_limits: (min = 0.0, max = 4800.0)
   rating: 200.0
   active_power: 0.0
   input_active_power_limits: (min = 0.0, max = 200.0)
   output_active_power_limits: (min = 0.0, max = 200.0)
   efficiency: (in = 0.85, out = 1.0)
   reactive_power: 0.0
   reactive_power_limits: nothing
   base_power: 200.0
   operation_cost: nothing
   services: 0-element Vector{Service}
   dynamic_injector: nothing
   ext: Dict{String, Any}()
   time_series_container: InfrastructureSystems.TimeSeriesContainer: 0
   internal: InfrastructureSystems.InfrastructureSystemsInternal

┌ Warning: SystemUnitSetting not defined, using NATURAL_UNITS for displaying device specification.
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/utils/print.jl:182


Now we add it to the system

In [16]:
PSY.add_component!(system, bat)
collect(get_components(GenericBattery, system))

1-element Vector{GenericBattery}:
 GenericBattery(Storage_24, true, Bus(102, Adams, BusTypes.PV = 3, -0.1364470483941135, 1.04783, (min = 0.95, max = 1.05), 138.0, Area(1, 0.0, 0.0, 0.0), LoadZone(12.0, 5.29, 1.08), Dict{String, Any}("neighbors" => Set([4, 6, 2, 1]))), PrimeMovers.BA = 1, 24.0, (min = 0.0, max = 48.0), 2.0, 0.0, (min = 0.0, max = 2.0), (min = 0.0, max = 2.0), (in = 0.85, out = 1.0), 0.0, nothing, 200.0, nothing, Service[], nothing, Dict{String, Any}())

Let's check the system is stable.

In [17]:
res = solve_powerflow(system)
res["bus_results"]

┌ Info: Validating connectivity with Goderya algorithm
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/utils/network_calculations/ybus_calculations.jl:308
┌ Info: The System has no islands
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/utils/network_calculations/ybus_calculations.jl:254
┌ Info: PowerFlow solve converged, the results are exported in DataFrames
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/utils/power_flow.jl:357
┌ Info: Voltages are exported in pu. Powers are exported in MW/MVAr.
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/utils/power_flow.jl:189


,bus_number,Vm,θ,P_gen,P_load,P_net,Q_gen,Q_load,Q_net
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,101,1.04777,-0.862755,0.0,108.0,-108.0,110.242,22.0,88.242
2,102,1.04783,-0.862282,0.0,97.0,-97.0,107.223,20.0,87.2227
3,103,0.983808,-0.783493,0.0,180.0,-180.0,0.0,37.0,-37.0
4,104,0.983463,-0.769658,0.0,74.0,-74.0,0.0,15.0,-15.0
5,105,1.00436,-0.784978,0.0,71.0,-71.0,0.0,14.0,-14.0
6,106,0.989394,-0.75807,0.0,136.0,-136.0,0.0,27.0211,-27.0211
7,107,1.03745,-0.729067,355.0,125.0,230.0,108.299,25.0,83.2991
8,108,0.987728,-0.743545,0.0,171.0,-171.0,0.0,35.0,-35.0
9,109,0.979653,-0.611487,0.0,175.0,-175.0,0.0,36.0,-36.0


Let's remove the component.

In [18]:
PSY.remove_component!(system, bat)
collect(get_components(GenericBattery, system))

GenericBattery[]

### Look at our current system

In our `gen_systems` folder, we created code to add 10 batteries. Let's take a look at it now and see how we can access *different* batteries. 

In [36]:
sys_name = "/lustre/eaglefs/projects/pvb/cju/gen_systems/data/rts_with_battery_060822_sys.json"
# sys_name = "/lustre/eaglefs/projects/pvb/5_bus_battery_testing/data/new_systems/pv=30_storagehours=10_wind=30/sys.json"
system_rts = System(sys_name, time_series_directory = "/tmp/scratch")
PSY.transform_single_time_series!(system_rts, 168, Hour(168))

collect(get_components(GenericBattery, system_rts))

┌ Info: Loaded time series from storage file existing=rts_with_battery_060822_sys_time_series_storage.h5 new=/tmp/scratch/jl_58gCcV compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
└ @ InfrastructureSystems /home/cju/.julia/packages/InfrastructureSystems/Oc56m/src/hdf5_time_series_storage.jl:100


GenericBattery[]

Let's check the system is stable.

In [37]:
# res = solve_powerflow(system_rts)
# res["bus_results"]

In [38]:
#= For 5 bus system
template_ed = ProblemTemplate(PowerSimulations.CopperPlatePowerModel)
set_device_model!(template_ed, ThermalStandard, ThermalDispatchNoMin)
set_device_model!(template_ed, PowerSystems.PowerLoad, StaticPowerLoad)
set_device_model!(template_ed, PowerSystems.GenericBattery, BookKeeping)

solver = optimizer_with_attributes(Xpress.Optimizer, "MIPRELSTOP" => 1e-4)
initial_time = DateTime("2024-01-01T00:00:00")

problem = DecisionModel(
             template_ed,
             system_rts,
             name = "SubProblem",
             optimizer = solver,
             warm_start = false,
             export_pwl_vars = true,
             initialize_model = false,
             initial_time = initial_time,
             horizon=24,
     )

# IMPORTANT! make sure to build the problem
directory ="./simulation_folder/"
build!(problem, output_dir = directory)
=#

Next, let's see how the batteries are represented in the system. First, let's generate a problem with this system.

From my discussion with Sourabh, we need to use a different problem problem since the system is not as "relaxed" as the 5 bus power system. 

In [63]:
# RTS has 3 regions (copper plot ignores Kirkoff's law, makes battery more valuable since harder to move energy around)
# PowerSimulations.DCPPowerModel, PowerSimulations.StandardPTDFModel instead of CopperPlatePowerModel
template_ed = ProblemTemplate(PowerSimulations.DCPPowerModel)

# other stuff I may have missed
# set_device_model!(template_ed, PowerSystems.ThermalStandard, ThermalStandardUnitCommitment)
# 
# DCPowerModel, Standar

# ThermalBasicUnitCommitment instead ThermalStandardUnitCommitment/ThermalDispatchNoMin
set_device_model!(template_ed, PowerSystemCaseBuilder.Line, StaticBranch)
set_device_model!(template_ed, PowerSystemCaseBuilder.Transformer2W, StaticBranch)
set_device_model!(template_ed, PowerSystemCaseBuilder.TapTransformer, StaticBranch)

set_device_model!(template_ed, PowerSystems.ThermalStandard, ThermalBasicUnitCommitment)
set_device_model!(template_ed, PowerSystems.RenewableDispatch, RenewableFullDispatch)
set_device_model!(template_ed, PowerSystems.PowerLoad, StaticPowerLoad)
set_device_model!(template_ed, PowerSystems.HydroDispatch, FixedOutput)
set_device_model!(template_ed, PowerSystems.GenericBattery, BookKeeping)
# Thermal is long sterm (RTS doesn't have that many)
# set_device_model!(template_ed, PowerSystems.HydroEnergyReservoir, HydroDispatchRunOfRiver)
set_device_model!(template_ed, PowerSystems.RenewableFix, FixedOutput)
set_service_model!(template_ed, VariableReserve{ReserveUp}, RangeReserve)
set_service_model!(template_ed, VariableReserve{ReserveDown}, RangeReserve)
# redundant
# set_network_model!(template_ed, NetworkModel(CopperPlatePowerModel))
##

solver = optimizer_with_attributes(Xpress.Optimizer, "MIPRELSTOP" => 1e-4)
initial_time = DateTime("2020-01-01T00:00:00")

problem = DecisionModel(
             template_ed,
             system_rts,
             name = "SubProblem",
             optimizer = solver,
             warm_start = false,
             export_pwl_vars = true,
             initialize_model = false,
             initial_time = initial_time,
             horizon=24,
     )

# IMPORTANT! make sure to build the problem
directory ="./simulation_folder/"
build!(problem, output_dir = directory)

BuildStatus.BUILT = 0

Let's now view the optimization problem.

Make sure we have the desired variables.

In [64]:
optimization_container = PSI.get_optimization_container(problem)
PSI.get_variable_keys(optimization_container)
# var_e = PSI.get_variable(optimization_container, EnergyVariable(), GenericBattery)

14-element Vector{PowerSimulations.VariableKey}:
 PowerSimulations.VariableKey{ActivePowerReserveVariable, VariableReserve{ReserveDown}}("Reg_Down")
 PowerSimulations.VariableKey{FlowActivePowerVariable, Line}("")
 PowerSimulations.VariableKey{ActivePowerReserveVariable, VariableReserve{ReserveUp}}("Spin_Up_R3")
 PowerSimulations.VariableKey{ActivePowerVariable, RenewableDispatch}("")
 PowerSimulations.VariableKey{VoltageAngle, Bus}("")
 PowerSimulations.VariableKey{ActivePowerReserveVariable, VariableReserve{ReserveUp}}("Spin_Up_R2")
 PowerSimulations.VariableKey{ActivePowerReserveVariable, VariableReserve{ReserveUp}}("Reg_Up")
 PowerSimulations.VariableKey{ActivePowerReserveVariable, VariableReserve{ReserveUp}}("Spin_Up_R1")
 PowerSimulations.VariableKey{StartVariable, ThermalStandard}("")
 PowerSimulations.VariableKey{PowerSimulations.PieceWiseLinearCostVariable, ThermalStandard}("")
 PowerSimulations.VariableKey{ActivePowerVariable, ThermalStandard}("")
 PowerSimulations.VariableKe

Is the problem solvable?

In [62]:
PSI.solve!(problem)

RunStatus.SUCCESSFUL = 0